In [1]:
import os
import tensorflow as tf

/Users/trinakarmakar/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
import keras 
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import Callback, TensorBoard, ModelCheckpoint

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)

In [ ]:
LOGDIR = './graphs/'

#hyperparameters
learning_rate = 0.01
batch_size = 1000
epochs = 10

#layers 
hl_1 = 1000
hl_2 = 500

#other parameters
input_size = 28*28
n_classes = 10

In [ ]:
sess = tf.Session()

with tf.name_scope('input'):
    images = tf.placeholder(tf.float32, [None, input_size], name="images")
    labels = tf.placeholder(tf.float32, [None, n_classes], name='labels')

def fc_layer(x, layer, size_out, activation=None):
    with tf.name_scope(layer):
        size_in = int(x.shape[1])
        W = tf.Variable(tf.random_normal([size_in, size_out]), name='weight')
        b = tf.Variable(tf.constant(-1, dtype=tf.float32, shape=[size_out]), name='bias')
        z = tf.add(tf.matmul(x, W), b)
        if activation:
            return activation(z)
        return z
    
fc_1 = fc_layer(images, "fc_1", hl_1, tf.nn.relu)
fc_2 = fc_layer(fc_1, "fc_2", hl_2, tf.nn.relu)
dropped = tf.nn.dropout(fc_2, keep_prob=0.9)
y = fc_layer(dropped, "output", n_classes)

In [ ]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=labels))
    tf.summary.scalar('loss', loss)

with tf.name_scope('optimizer'):
    train = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

with tf.name_scope('evaluation'):
    correct = tf.equal( tf.argmax(y, 1), tf.argmax(labels, 1) )
    accuracy = tf.reduce_mean( tf.cast(correct, dtype=tf.float32) )
    tf.summary.scalar('accuracy', accuracy)

In [ ]:
#using keras
model = Sequential()
model.add(Dense(1000, input_dim=input_size, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dropout(rate=0.9))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
class Callback(object):
    """Abstract base class used to build new callbacks.

    # Properties
        params: dict. Training parameters
            (eg. verbosity, batch size, number of epochs...).
        model: instance of `keras.models.Model`.
            Reference of the model being trained.

    The `logs` dictionary that callback methods
    take as argument will contain keys for quantities relevant to
    the current batch or epoch.

    Currently, the `.fit()` method of the `Sequential` model class
    will include the following quantities in the `logs` that
    it passes to its callbacks:

        on_epoch_end: logs include `acc` and `loss`, and
            optionally include `val_loss`
            (if validation is enabled in `fit`), and `val_acc`
            (if validation and accuracy monitoring are enabled).
        on_batch_begin: logs include `size`,
            the number of samples in the current batch.
        on_batch_end: logs include `loss`, and optionally `acc`
            (if accuracy monitoring is enabled).
    """

    def __init__(self):
        self.validation_data = None

    def set_params(self, params):
        self.params = params

    def set_model(self, model):
        self.model = model

    def on_epoch_begin(self, epoch, logs=None):
        pass

    def on_epoch_end(self, epoch, logs=None):
        pass

    def on_batch_begin(self, batch, logs=None):
        pass

    def on_batch_end(self, batch, logs=None):
        pass

    def on_train_begin(self, logs=None):
        pass

    def on_train_end(self, logs=None):
        pass

In [ ]:
def on_epoch_end(self, epoch, logs=None):
    logs = logs or {}

    if not self.validation_data and self.histogram_freq:
        raise ValueError('If printing histograms, validation_data must be '
                         'provided, and cannot be a generator.')
    if self.validation_data and self.histogram_freq:
        if epoch % self.histogram_freq == 0:

            val_data = self.validation_data
            tensors = (self.model.inputs +
                       self.model.targets +
                       self.model.sample_weights)

            if self.model.uses_learning_phase:
                tensors += [K.learning_phase()]

            assert len(val_data) == len(tensors)
            val_size = val_data[0].shape[0]
            i = 0
            while i < val_size:
                step = min(self.batch_size, val_size - i)
                if self.model.uses_learning_phase:
                    # do not slice the learning phase
                    batch_val = [x[i:i + step] for x in val_data[:-1]]
                    batch_val.append(val_data[-1])
                else:
                    batch_val = [x[i:i + step] for x in val_data]
                assert len(batch_val) == len(tensors)
                feed_dict = dict(zip(tensors, batch_val))
                result = self.sess.run([self.merged], feed_dict=feed_dict)
                summary_str = result[0]
                self.writer.add_summary(summary_str, epoch)
                i += self.batch_size

    if self.embeddings_freq and self.embeddings_ckpt_path:
        if epoch % self.embeddings_freq == 0:
            self.saver.save(self.sess,
                            self.embeddings_ckpt_path,
                            epoch)

    for name, value in logs.items():
        if name in ['batch', 'size']:
            continue
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value.item()
        summary_value.tag = name
        self.writer.add_summary(summary, epoch)
    self.writer.flush()

In [ ]:
for name, value in logs.items():
    if name in ['batch', 'size']:
        continue
    summary = tf.Summary()
    summary_value = summary.value.add()
    summary_value.simple_value = value.item()
    summary_value.tag = name
    self.writer.add_summary(summary, epoch)
self.writer.flush()

In [ ]:
cb = TensorBoard()
history_callback = model.fit(
    x=mnist.train.images, 
    y=mnist.train.labels, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE,
    callbacks=[cb])

In [ ]:
history_callback.__dict__

In [ ]:
score = model.evaluate(
    x=mnist.test.images,
    y=mnist.test.labels)

print("score = ", score)

In [ ]:
#Batch-wise plots
class Batched_TensorBoard(TensorBoard):

    def __init__(self):
        self.log_dir = "./log_dir"
        self.batch_writer = tf.summary.FileWriter(self.log_dir) # Created here as site-packages/keras/callback.py
        self.step = 0 # Initialization
        super().__init__(self.log_dir) # Execute TensorBoard's constructor, passing the log directory


    def on_batch_end(self, batch, logs={}):
        """Called after every batch"""

        for name, value in logs.items():
            if name in ['acc', 'loss']:
                summary = tf.Summary()
                summary_value = summary.value.add() #Empty
                summary_value.simple_value = value.item() # 0.87 (Accuracy Value)
                summary_value.tag = name #if "acc", tag = "accuracy" for more defined tags on the tensorboard
                self.batch_writer.add_summary(summary, self.step) 
    
        self.batch_writer.flush()
        self.step += 1 # Iterated over every batch

In [ ]:
cb = Batched_TensorBoard()

history_callback = model.fit(
    x=mnist.train.images, 
    y=mnist.train.labels, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE,
    callbacks=[cb])

In [ ]:
history_callback.__dict__